# [Dev notebook]
## How does STGL work with TICC data?

In [ ]:
%matplotlib inline

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from imp import reload
from time import time

from sklearn.utils.extmath import squared_norm
from sklearn.base import clone
from sklearn.covariance import empirical_covariance, EmpiricalCovariance, GraphicalLasso
from sklearn.cluster.hierarchical import AgglomerativeClustering
from scipy import linalg
from sklearn.metrics import v_measure_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared
from sklearn.pipeline import make_pipeline, Pipeline

from regain.covariance import (time_graphical_lasso_, latent_time_graphical_lasso_,
                               kernel_time_graphical_lasso_, kernel_latent_time_graphical_lasso_)

from regain.datasets import kernels
from regain.utils import structure_error, error_norm_time, normalize_matrix

from ticc import TICC_solver
reload(TICC_solver)

from tqdm import tqdm

import local_utils
reload(local_utils)
from local_utils import *

from regain.bayesian import wishart_process_, sampling, stats; reload(wishart_process_)
idx = pd.IndexSlice

# # print the inv matrix
# res = sns.clustermap(data.inv, row_cluster=False, col_cluster=False)
# ax = res.ax_heatmap
# for i in range(n_samples * len(clusters)):
#     ax.axvline((i + 1) * n_dim)
#     ax.axhline((i + 1) * n_dim)

# cov = []
# for c in np.unique(y):
#     idx = y == c
#     cov.append(empirical_covariance(X[idx]))
# cov = np.asarray(cov)

# kernel = np.zeros((cov.shape[0], cov.shape[0]))
# for i in range(cov.shape[0]):
#     for j in range(i + 1, cov.shape[0]):
#         kernel[i, j] = kernel[j, i] = (np.linalg.norm((cov[i]) - (cov[j])))

# mm = np.sum(np.abs(kernel), axis=1)
# kernel += np.eye(cov.shape[0]) * mm

# normalize_matrix(kernel)


In [ ]:
alpha = 1
beta = 2
eta = 4

stgl = kernel_time_graphical_lasso_.SimilarityTimeGraphicalLasso(
    alpha=alpha, beta=beta, max_iter=500, psi='l1',
    max_iter_ext=10, eps=1e-1)

sltgl = kernel_latent_time_graphical_lasso_.SimilarityLatentTimeGraphicalLasso(
    alpha=alpha, tau=3, beta=beta, eta=eta, max_iter=500, psi='l1',
    max_iter_ext=10, eps=1e-1)
clust = AgglomerativeClustering(affinity='precomputed', linkage='complete')

In [ ]:
# from sklearn.model_selection import StratifiedShuffleSplit
# from skopt import searchcv; from imp import reload; reload(searchcv)
# from skopt.space import Real, Categorical, Integer
# from sklearn.datasets import load_iris
# from sklearn.svm import SVC 
# from sklearn.model_selection import train_test_split

# domain = {'alpha': Real(1e-1, 1e0, prior='log-uniform'),
#           'tau': Real(1e-1, 1e+1, prior='log-uniform'),
#           'beta': Integer(1, 60)}
    
# cv = StratifiedShuffleSplit(10, test_size=0.2)
    
# bscv = searchcv.BayesSearchCV(sltgl, domain, n_iter=32, cv=cv)

# bscv.fit(X, y)

In [ ]:
partitions = [5, 10, 50, 100, 500]
n_splits = 5

n_dim = 100
w_size = 2
n_samples = 500

In [ ]:
# (a) and (b) experiment
shuffle = False

np.random.seed(42)
vs, ne, se = {}, {}, {}
datas = {}
for i in partitions:
    for j in tqdm(range(n_splits)):
        np.random.seed((i+j+3)*n_dim)
        n_clusts = np.random.randint(10) + 5
        data = kernels.make_cluster_representative(
            n_clusters=n_clusts, n_dim=n_dim, n_samples=i, shuffle=shuffle)
        X, y = data.X, data.y
        n_times = np.unique(y).size
        thetas_true = np.array([data.precs[l] for l in y])
        thetas_true_sparse = np.array([data.sparse_precs[l] for l in y])
        labels_true = data.id_cluster #[::(len(clusters) * i // n_times)]

        
        # WP + similarity
        wp = wishart_process_.WishartProcess(kernel='ess', verbose=2, learn_ell=False,
                                            n_iter=200).fit(X, y)
        tic = time()
        wp.fit(X, y)        
        tac = time() - tic
        tgl_prec_sims = kernel_time_graphical_lasso_.precision_similarity(wp.precision_, psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(tgl_prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        mdl = wp
        obs_precs = np.array([mdl.precision_[l] for l in y])
        obs_precs_sparse = np.array([mdl.precision_[l] for l in y])
        
        set_results(vs, mdl, 'wp', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
        
        
        scales = np.unique(np.diff(np.where(data.cluster_series == 0)))
        scales = scales[scales > 1]
        kernel = RBF(1)
        for s in scales:
            kernel += ExpSineSquared(periodicity=s, length_scale=0.5) * 10
        kern = kernel(np.arange(np.unique(y).size)[:, None])
        
        # KTGL + similarity
        mdl = kernel_time_graphical_lasso_.KernelTimeGraphicalLasso(
            alpha=1, max_iter=500, psi='l1', kernel=kern)
        
        tic = time()
        mdl.fit(X, y)        
        psi = kernel_time_graphical_lasso_.check_norm_prox(mdl.psi)[0]
        tgl_prec_sims = kernel_time_graphical_lasso_.precision_similarity(mdl.get_observed_precision(), psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(tgl_prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        tac = time() - tic
        obs_precs = np.array([mdl.get_observed_precision()[l] for l in y])
        obs_precs_sparse = np.array([mdl.get_precision()[l] for l in y])
        
        set_results(vs, mdl, 'ktgl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
        
        
        # STGL
        pipe_stgl = Pipeline([('stgl', clone(stgl)), ('clust', clone(clust))])
        pipe_stgl.set_params(stgl__n_clusters=np.unique(labels_true).size,
                             clust__n_clusters=np.unique(labels_true).size)
        tic = time()
        labels_pred = pipe_stgl.fit_predict(X, y)
        tac = time() - tic
        labels_pred = np.repeat(labels_pred, i)
        
        mdl = pipe_stgl['stgl']
        obs_precs = np.array([mdl.get_observed_precision()[l] for l in y])
        obs_precs_sparse = np.array([mdl.get_precision()[l] for l in y])
        
        set_results(vs, pipe_stgl, 'stgl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
        
        # GraphicalLasso + similarity
        gl = GraphicalLasso(alpha=alpha)
        tic = time()
        mdl = [clone(gl).fit(X[y == c]) for c in np.unique(y)]
        tac = time() - tic
        precisions = np.array([m.precision_ for m in mdl])
        psi = kernel_time_graphical_lasso_.check_norm_prox('l1')[0]
        prec_sims = kernel_time_graphical_lasso_.precision_similarity(precisions, psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        obs_precs = np.array([precisions[l] for l in y])
        obs_precs_sparse = np.array([precisions[l] for l in y])
        
        set_results(vs, mdl, 'gl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
        
        # TGL + similarity
        tgl = time_graphical_lasso_.TimeGraphicalLasso(alpha=alpha, beta=beta, max_iter=500, psi='l1')
        tic = time()
        tgl.fit(X, y)        
        tac = time() - tic
        psi = kernel_time_graphical_lasso_.check_norm_prox(tgl.psi)[0]
        tgl_prec_sims = kernel_time_graphical_lasso_.precision_similarity(tgl.get_observed_precision(), psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(tgl_prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        mdl = tgl
        obs_precs = np.array([mdl.get_observed_precision()[l] for l in y])
        obs_precs_sparse = np.array([mdl.get_precision()[l] for l in y])
        
        set_results(vs, mdl, 'tgl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)

    
        ticc = TICC_solver.TICC(number_of_clusters=np.unique(labels_true).size, window_size=1,
                                lambda_parameter=alpha)
        tic = time()
        try:
            ticc.fit(X)
            
        except Exception as e:
            print("ticc error. %s" % str(e), end=' ')
            ticc.labels_ = np.zeros_like(labels_true)
            ticc.precision_ = {0: np.eye(X.shape[1])}
            
        tac = time() - tic
        
        obs_precs = np.array([ticc.precision_[l] for l in ticc.labels_])
        obs_precs_sparse = obs_precs
        
        set_results(vs, ticc, 'ticc', i, labels_true, ticc.labels_, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)

In [ ]:
from regain import utils
# utils.save_pickle(datas, 'datas_1308_noshuffle_100.pkl')
utils.save_pickle(vs, 'results_representative_1308_100_noshuffle.pkl')

In [ ]:
# (a) and (b) experiment
shuffle = True

np.random.seed(42)
vs, ne, se = {}, {}, {}
datas = {}
for i in partitions:
    for j in tqdm(range(n_splits)):
        np.random.seed((i+j+3)*n_dim)
        n_clusts = np.random.randint(10) + 5
        data = kernels.make_cluster_representative(
            n_clusters=n_clusts, n_dim=n_dim, n_samples=i, shuffle=shuffle)
        X, y = data.X, data.y
        n_times = np.unique(y).size
        thetas_true = np.array([data.precs[l] for l in y])
        thetas_true_sparse = np.array([data.sparse_precs[l] for l in y])
        labels_true = data.id_cluster #[::(len(clusters) * i // n_times)]

        
        # WP + similarity
        wp = wishart_process_.WishartProcess(kernel='ess', verbose=2, learn_ell=False,
                                            n_iter=200).fit(X, y)
        tic = time()
        wp.fit(X, y)        
        tac = time() - tic
        tgl_prec_sims = kernel_time_graphical_lasso_.precision_similarity(wp.precision_, psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(tgl_prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        mdl = wp
        obs_precs = np.array([mdl.precision_[l] for l in y])
        obs_precs_sparse = np.array([mdl.precision_[l] for l in y])
        
        set_results(vs, mdl, 'wp', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
        
        
        scales = np.unique(np.diff(np.where(data.cluster_series == 0)))
        scales = scales[scales > 1]
        kernel = RBF(1)
        for s in scales:
            kernel += ExpSineSquared(periodicity=s, length_scale=0.5) * 10
        kern = kernel(np.arange(np.unique(y).size)[:, None])
        
        # KTGL + similarity
        mdl = kernel_time_graphical_lasso_.KernelTimeGraphicalLasso(
            alpha=1, max_iter=500, psi='l1', kernel=kern)
        
        tic = time()
        mdl.fit(X, y)        
        psi = kernel_time_graphical_lasso_.check_norm_prox(mdl.psi)[0]
        tgl_prec_sims = kernel_time_graphical_lasso_.precision_similarity(mdl.get_observed_precision(), psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(tgl_prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        tac = time() - tic
        obs_precs = np.array([mdl.get_observed_precision()[l] for l in y])
        obs_precs_sparse = np.array([mdl.get_precision()[l] for l in y])
        
        set_results(vs, mdl, 'ktgl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
        
        
        # STGL
        pipe_stgl = Pipeline([('stgl', clone(stgl)), ('clust', clone(clust))])
        pipe_stgl.set_params(stgl__n_clusters=np.unique(labels_true).size,
                             clust__n_clusters=np.unique(labels_true).size)
        tic = time()
        labels_pred = pipe_stgl.fit_predict(X, y)
        tac = time() - tic
        labels_pred = np.repeat(labels_pred, i)
        
        mdl = pipe_stgl['stgl']
        obs_precs = np.array([mdl.get_observed_precision()[l] for l in y])
        obs_precs_sparse = np.array([mdl.get_precision()[l] for l in y])
        
        set_results(vs, pipe_stgl, 'stgl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
        
        # GraphicalLasso + similarity
        gl = GraphicalLasso(alpha=alpha)
        tic = time()
        mdl = [clone(gl).fit(X[y == c]) for c in np.unique(y)]
        tac = time() - tic
        precisions = np.array([m.precision_ for m in mdl])
        psi = kernel_time_graphical_lasso_.check_norm_prox('l1')[0]
        prec_sims = kernel_time_graphical_lasso_.precision_similarity(precisions, psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        obs_precs = np.array([precisions[l] for l in y])
        obs_precs_sparse = np.array([precisions[l] for l in y])
        
        set_results(vs, mdl, 'gl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
        
        # TGL + similarity
        tgl = time_graphical_lasso_.TimeGraphicalLasso(alpha=alpha, beta=beta, max_iter=500, psi='l1')
        tic = time()
        tgl.fit(X, y)        
        tac = time() - tic
        psi = kernel_time_graphical_lasso_.check_norm_prox(tgl.psi)[0]
        tgl_prec_sims = kernel_time_graphical_lasso_.precision_similarity(tgl.get_observed_precision(), psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(tgl_prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        mdl = tgl
        obs_precs = np.array([mdl.get_observed_precision()[l] for l in y])
        obs_precs_sparse = np.array([mdl.get_precision()[l] for l in y])
        
        set_results(vs, mdl, 'tgl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)

    
        ticc = TICC_solver.TICC(number_of_clusters=np.unique(labels_true).size, window_size=1,
                                lambda_parameter=alpha)
        tic = time()
        try:
            ticc.fit(X)
            
        except Exception as e:
            print("ticc error. %s" % str(e), end=' ')
            ticc.labels_ = np.zeros_like(labels_true)
            ticc.precision_ = {0: np.eye(X.shape[1])}
            
        tac = time() - tic
        
        obs_precs = np.array([ticc.precision_[l] for l in ticc.labels_])
        obs_precs_sparse = obs_precs
        
        set_results(vs, ticc, 'ticc', i, labels_true, ticc.labels_, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)

In [ ]:
from regain import utils
# utils.save_pickle(datas, 'datas_1308_noshuffle_100.pkl')
utils.save_pickle(vs, 'results_representative_1308_100_shuffle.pkl')

In [ ]:
# (c) experiment - LATENT

np.random.seed(42)
vs, ne, se = {}, {}, {}
datas = {}
for i in partitions[::-1]:
    for j in (range(n_splits)):
        np.random.seed((i+j+3)*n_dim)
        n_clusts = np.random.randint(10) + 5
        
        data = kernels.make_ticc_dataset_v3(
            clusters=np.random.choice(list(range(n_clusts)), size=20),
            w_size=w_size, n_dim=n_dim, n_samples=i)
        X, y = data.X, data.y
        n_times = np.unique(y).size
        thetas_true = np.array([data.precs[l] for l in y])
        thetas_true_sparse = np.array([data.sparse_precs[l] for l in y])
        labels_true = data.id_cluster #[::(len(clusters) * i // n_times)]

        scales = np.unique(np.diff(np.where(data.id_cluster == 0)))
        scales = scales[scales > 1]
        kernel = RBF(1)
        for s in scales:
            kernel += ExpSineSquared(periodicity=s, length_scale=0.5) * 10
        kern = kernel(np.arange(np.unique(y).size)[:, None])
        
        kern_phi = np.eye(np.unique(y).size)
        np.fill_diagonal(kern_phi[:, 1:], eta)
        np.fill_diagonal(kern_phi[1:], eta)
        
        # KLTGL + similarity
        mdl = kernel_latent_time_graphical_lasso_.KernelLatentTimeGraphicalLasso(
            alpha=alpha, tau=5, max_iter=500, psi='l1',
            kernel_psi=kern,
            kernel_phi=kern_phi)
        tic = time()
        mdl.fit(X, y)
        psi = kernel_latent_time_graphical_lasso_.check_norm_prox(mdl.psi)[0]
        ltgl_prec_sims = kernel_time_graphical_lasso_.precision_similarity(mdl.get_observed_precision(), psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(ltgl_prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        tac = time() - tic
        
        obs_precs = np.array([mdl.get_observed_precision()[l] for l in y])
        obs_precs_sparse = np.array([mdl.get_precision()[l] for l in y])
        
        set_results(vs, mdl, 'kltgl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)

 
        # SLTGL
        pipe_sltgl = Pipeline([('sltgl', clone(sltgl)), ('clust', clone(clust))])
        pipe_sltgl.set_params(sltgl__n_clusters=np.unique(labels_true).size,
                              sltgl__tau=5,
                              clust__n_clusters=np.unique(labels_true).size)
        tic = time()
        labels_pred = pipe_sltgl.fit_predict(X, y)
        tac = time() - tic
        labels_pred = np.repeat(labels_pred, i)
        
        mdl = pipe_sltgl['sltgl']
        obs_precs = np.array([mdl.get_observed_precision()[l] for l in y])
        obs_precs_sparse = np.array([mdl.get_precision()[l] for l in y])
        
        set_results(vs, pipe_sltgl, 'sltgl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
        
        
        # LTGL + similarity
        mdl = latent_time_graphical_lasso_.LatentTimeGraphicalLasso(
            alpha=alpha, beta=beta, tau=5, eta=eta, max_iter=500, psi='l1')
        tic = time()
        mdl.fit(X, y)
        tac = time() - tic
        psi = kernel_latent_time_graphical_lasso_.check_norm_prox(mdl.psi)[0]
        ltgl_prec_sims = kernel_time_graphical_lasso_.precision_similarity(mdl.get_observed_precision(), psi)
        labels_pred = clust.set_params(n_clusters=np.unique(labels_true).size).fit_predict(ltgl_prec_sims)
        labels_pred = np.repeat(labels_pred, i)
        
        obs_precs = np.array([mdl.get_observed_precision()[l] for l in y])
        obs_precs_sparse = np.array([mdl.get_precision()[l] for l in y])
        
        set_results(vs, mdl, 'ltgl', i, labels_true, labels_pred, 
                    thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
                
#         # TICC
#         ticc = TICC_solver.TICC(number_of_clusters=np.unique(labels_true).size, window_size=1,
#                                 lambda_parameter=alpha)
#         tic = time()
#         try:
#             ticc.fit(X)
            
#         except Exception as e:
#             print("ticc error. %s" % str(e), end=' ')
#             ticc.labels_ = np.zeros_like(labels_true)
#             ticc.precision_ = {0: np.eye(X.shape[1])}
            
#         tac = time() - tic
        
#         obs_precs = np.array([ticc.precision_[l] for l in ticc.labels_])
#         obs_precs_sparse = obs_precs
        
#         set_results(vs, ticc, 'ticc', i, labels_true, ticc.labels_, 
#                     thetas_true_sparse, thetas_true, obs_precs_sparse, obs_precs, tac)
    from regain import utils
    # utils.save_pickle(datas, 'datas_1308_noshuffle_100.pkl')
    utils.save_pickle(vs, 'results_representative_1508_100_ticc.pkl')

## Loading Results

In [ ]:
import local_utils; reload(local_utils)
from local_utils import *

from regain import utils
vs_0 = utils.load_pickle('/Users/federicot/Downloads/results_representative_1308_100_noshuffle.pkl')
vs_1 = utils.load_pickle('/Users/federicot/Downloads/results_representative_1308_100_shuffle.pkl')
vs_2 = utils.load_pickle('/Users/federicot/Downloads/results_representative_1508_100_ticc.pkl')

vss = [vs_0, vs_1, vs_2]

In [ ]:
only_in_plot = ['time']
in_plot = only_in_plot + ['mcc', 'error_norm', 'v_measure']

In [ ]:
print_latent = False
latent_mdls = ['KLTGL', 'SLTGL', 'LTGL', 'TICC']

other_measures = ['time', 'error_norm', 'v_measure']
mm = ['balanced_accuracy', 'average_precision', 'mcc'] + other_measures
mm = np.array(mm).reshape(2, 3)
mapping_names = dict(zip(['KTGL', 'STGL', 'KLTGL', 'SLTGL'], ['TGL$_\kappa$', 'TGL$_P$', 'LTGL$_\kappa$', 'LTGL$_P$']))
    
melt_exps = []
for it, vs in enumerate(vss):
    new_r = convert_dict_to_df(vs, max_samples=1000)
    try:
        ensure_ticc_valid(new_r)
    except:
        pass

    melt_0 = pd.melt(new_r.reset_index(), id_vars=['method', 'samples'], value_vars=['structure_error'])
    for k in melt_0.value[0]:
        melt_0[k] = melt_0.value.apply(lambda x: x[k] if x else np.nan)

    melts = [melt_0] + [
        pd.melt(new_r.reset_index(), id_vars=['method', 'samples'], value_vars=[val], value_name=val)
        for val in other_measures
    ]
    melts = [
        melt_.set_index(['method', 'samples', melt_.groupby(['method', 'samples']).cumcount()])
        for melt_ in melts
    ]

    melt = pd.concat(melts, axis=1).sort_index(level=2).reset_index(level=2, drop=True).reset_index()

    if not print_latent and it < 2:
        melt = melt[~melt.method.isin(latent_mdls + ['ECOV'])]
    else:
        melt = melt[melt.method.isin(latent_mdls + ['TICC'])]
    
    melt.method = pd.Categorical(melt.method, ["GL", 'WP', "TGL", 'LTGL', 'KTGL', 'KLTGL', 'STGL',  'SLTGL'])
    melt = melt.sort_values('method')
    melt = melt.applymap(lambda x: x if not isinstance(x, str) else mapping_names.get(x, x))
    melt_exps.append(melt)

In [ ]:
for exp, melt in zip(['noshuffle', 'shuffle', 'ticc'], melt_exps):
    res_table = pd.DataFrame()
    for m in mm.flatten():
        if m in only_in_plot: continue
    #     res_table[m] = melt[m].apply(format_3f)
    #     res_table[m] = melt.groupby('method')[m].apply(format_3f)
        res_table[m] = melt.groupby(['method', 'samples'][::-1])[m].apply(format_3f)

    res_table[''] = np.where(res_table.index.levels[1].str.endswith('TGL$_\kappa$'),
                             '\\ldelim\\{{2}{5.5mm}[Ours]', '').tolist() * res_table.index.levels[0].size
    res_table = res_table.reset_index().set_index(['samples', '', 'method'])

    display(res_table.style.apply(highlight_max_std, axis=None))
    
    res_table.loc[idx[50, :]].applymap(lambda x: '$%s$'%x).to_latex(
        "results_100_50s_{}.tex".format(exp), escape=False)

In [ ]:
palette = dict(zip(["TGL", 'TGL$_\kappa$', 'TGL$_P$', "GL", 'WP'], sns.color_palette()))
palette.update(dict(zip(['LTGL', 'LTGL$_\kappa$', 'LTGL$_P$'], sns.color_palette())))

In [ ]:
# melt_idx = melt.set_index(['method', 'samples'])
# for m in mm.flatten():
#     melt_idx[m] -= melt_idx.loc[idx['TGL', :], m].groupby('samples').mean()

plt.close('all')
hh, ll = [], []
f, ax = plt.subplots(len(melt_exps), len(in_plot), figsize=(16,7),
                     sharex=True, sharey=False, squeeze=False)#, gridspec_kw=dict(wspace=0.4, hspace=0.1))

for c, measure in enumerate(in_plot):
    for r, melt in enumerate(melt_exps):
        ax[r,c] = sns.pointplot(
            x='samples', y=measure, data=melt,
            hue='method', ax=ax[r,c], legend=False, palette=palette)
        if c < len(in_plot) - 1:
            ax[r,c].legend_ = None
        else:
            lgd = ax[r,c].legend(title="Exp ({})".format(dict(zip(range(len(melt_exps)), list('abc')))[r]),
                                 bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
            ll.append(lgd)
        
        if r < len(melt_exps) - 1:
            ax[r,c].set_xlabel('')
        if r == 0:
            ax[r,c].set_title(
                dict(zip(in_plot, ['Time', 'MCC', 'MSE', 'V-measure']))[measure]
            )
        ax[r,c].set_ylabel('')
        
        # ax[r,c].axhline(0, c='k', ls='--')
        if measure in ['tp','fp','fn','tn', 'time', 'error_norm']:
            ax[r,c].set_yscale('log')

# for handles, labels in zip(hh, ll):
#     lgd = f.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
f.tight_layout()

In [ ]:
f.savefig('/Users/federicot/src/fdtomasi/wsdm20/images/plot_1308_all.pdf', 
          transparent=True, bbox_extra_artists=ll, bbox_inches='tight', dpi=600)
#           transparent=True, bbox_extra_artists=(lgd,), bbox_inches='tight', dpi=600)

In [ ]:
# melt_idx = melt.set_index(['method', 'samples'])
# for m in mm.flatten():
#     melt_idx[m] -= melt_idx.loc[idx['TGL', :], m].groupby('samples').mean()

plt.close('all')
f, ax = plt.subplots(mm.shape[0], mm.shape[1], figsize=(20,5), sharex=True, squeeze=False)
for r in range(mm.shape[0]):
    for c in range(mm.shape[1]):
        ax[r,c] = sns.pointplot(
            x='samples', y=mm[r,c], data=melt,
            hue='method', ax=ax[r,c])
        melt[mm[r,c]] = melt[mm[r,c]].astype(float)
        
        # ax[r,c].axhline(0, c='k', ls='--')
        if mm[r,c] in ['tp','fp','fn','tn', 'time', 'error_norm']:
            ax[r,c].set_yscale('log')
    
f.tight_layout()
f

In [ ]:
f.savefig('plot_1308_shuffle.pdf')

In [ ]:
vss = {}
for k in vs:
    if k[1] < 1000:
        vss[k] = vs[k]

neww = {}
for k, v in vss.items():
    new = {}
    for kk, vv in v.items():
        new.update({(kk, i): vvv for i, vvv in enumerate(vv)})
    neww[k] = new

res_df = pd.DataFrame(neww)
res_df.index.name = ('measure', 'iter')

rr = res_df.T.reset_index()
rr = rr.rename(columns={'level_0': 'method', 'level_1': 'samples'})
rr.method = rr.method.str.upper()

new_r = rr.set_index(['method', 'samples'])

for i in range(10):
    new_r['valid', i] = True

for r in new_r.loc['TICC', 'model'].iterrows():
    sampl = r[0]
    for k, rrr in (r[1].iteritems()):
        if np.alltrue(rrr.labels_ == np.zeros_like(rrr.labels_)):
            new_r.loc[('TICC', sampl), ('structure_error', k)] = None

rr = new_r.reset_index()

latent_mdls = ['KLTGL', 'SLTGL', 'LTGL']#, 'TICC']
print_latent = False

melt = pd.melt(rr, id_vars=['method', 'samples'], value_vars=['structure_error'])

for k in melt.value[0]:
    melt[k] = melt.value.apply(lambda x: x[k] if x else np.nan)

# mm = np.array(list(melt.value[0].keys())).reshape(7,3)
mm = np.array(['balanced_accuracy', 'average_precision', 'mcc'])

if not print_latent:
    melt = melt[~melt.method.isin(latent_mdls + ['ECOV'])]
else:
    melt = melt[melt.method.isin(latent_mdls + ['TICC'])]

melt_idx = melt.set_index(['method', 'samples'])
# for m in mm.flatten():
#     melt_idx[m] -= melt_idx.loc[idx['TGL', :], m].groupby('samples').mean()

res_table = pd.DataFrame()
for m in mm.flatten():
    res_table[m] = melt.groupby('method')[m].apply(format_3f)

# plt.close('all')
# f, ax = plt.subplots(mm.shape[0], mm.shape[1], figsize=(20,10), sharex=True)
# for r in range(mm.shape[0]):
#     for c in range(mm.shape[1]):
#         ax[r,c] = sns.pointplot(x='samples', y=mm[r,c], data=melt_idx.reset_index(), hue='method', ax=ax[r,c])
# #         ax[r,c].axhline(0, c='k', ls='--')
#         if mm[r,c] in ['tp','fp','fn','tn']:
#             ax[r,c].set_yscale('log')

# f.tight_layout()
# f
# f.savefig('structure_error_no_shuffle.pdf')

plt.close('all')
f, ax = plt.subplots(1,3, figsize=(15,5), sharex=True)
for i, val in enumerate(['time', 'error_norm', 'v_measure']):
    melt = pd.melt(rr, id_vars=['method', 'samples'], value_vars=[val], value_name=val)
    if not print_latent:
        melt = melt[~melt.method.isin(latent_mdls + ['ECOV'])]
    else:
        melt = melt[melt.method.isin(latent_mdls + ['TICC'])]
    res_t = melt.groupby('method')[val].apply(format_3f)
    res_table[val] = res_t
    
    # melt = melt[~(melt.method.isin(['TICC',]) & melt.samples != 500)]
    
    melt_idx = melt.set_index(['method', 'samples'])
    
    melt_idx[val] = melt_idx[val].astype(float)
#     melt_idx[val] -= melt_idx.loc[idx['TGL', :], val].groupby('samples').mean()
    melt_idx.loc[idx['TICC', [5, 10, 50, 100]], val] = np.nan

    ax[i] = sns.pointplot(x='samples', y=val, data=melt_idx.reset_index(), hue='method', ax=ax[i])
    if val in ['time', 'error_norm']:
        ax[i].set_yscale('log')
f.tight_layout()

res_table.style.apply(highlight_max_std)

In [ ]:
def enclose_num(x):
    mean = x.split(' ')[0]
    bm = False
    if 'bm' in mean:
        bm = True
        mean = mean.split('{')[-1]
    std = x.split(' ')[-1].split(')')[0]
    r = '\expn{%s}{%s} (\pm \expn{%s}{%s})' % (*mean.split('e'), *std.split('e'))
    if bm:
        return '\\bm{%s}' % r
    return r

In [ ]:
# res_table.applymap(enclose_num).applymap(lambda x: '$%s$'%x).to_latex("results_ticc.tex", escape=False)
res_table.applymap(lambda x: '$%s$'%x).to_latex("results_noshuffle_3f_5samples.tex", escape=False)

In [ ]:
from regain.plotting import results
mdls = ['stgl', 'sltgl', 'tgl', 'ltgl', 'ticc']

In [ ]:
mdls = np.unique([x[0] for x in vs])

In [ ]:
# datas = utils.load_pickle('/Users/federicot/Downloads/datas_0508_shuffle.pkl')

vs = utils.load_pickle('/Users/federicot/Downloads/results_representative_0508_ticc_2.pkl')

# for f in ['/Users/federicot/Downloads/results_representative_0508_shuffle.pkl',]:
# #           '/Users/federicot/Downloads/results_representative_0508_ticc.pkl']:
#     vs.update(utils.load_pickle(f))

In [ ]:
data_noshuffle = utils.load_pickle('/Users/federicot/Downloads/datas_0508_ticc_2.pkl')

In [ ]:
for k, k1 in zip(datas, data_noshuffle):
    datas[k]['data'] += data_noshuffle[k]['data']

In [ ]:
from itertools import product

In [ ]:
dd = pd.DataFrame(index=product(['no_shuffle', 'shuffle'], datas.keys()), columns=range(5))

In [ ]:
trues = np.vstack(np.array([[d['precs'] for d in datas[k]['data']] for k in datas]))
trues_noshuffle = np.vstack(np.array([[d['precs'] for d in data_noshuffle[k]['data']] for k in data_noshuffle]))

In [ ]:
all_trues = np.vstack((trues, trues_noshuffle))
all_trues.shape

In [ ]:
pp = {}
for mdl in mdls:
    pr = []
    vs[mdl] = [vs[(mdl, n)]['model'] for n in data_noshuffle]
    if mdl == 'ticc': continue
#         vs[mdl] = np.vstack(vs[mdl])
#         for x in vs[mdl]:
#             precision_ = [np.array([xx.precision_[l] for l in xx.labels_]) for xx in x]
#             pr.append(np.array(np.abs(precision_)))
            
    if mdl in ['ecov', 'gl']:
        for x in vs[mdl]:
            prr = []
            for xx in x:
                precision_ = np.array([xxx.precision_ for xxx in xx])
                prr.append(np.array(np.abs(precision_)))
            pr.append(prr)
            
#         pp[mdl] = np.array([np.abs(np.array([xx.precision_ for xx in x]))
#                    for x in vs[mdl]])
    else:
#             pr.append(np.array([xx.precision_ for xx in x]))
        if not mdl.startswith('s'):
            vs[mdl] = np.vstack(vs[mdl])
            
        for x in vs[mdl]:
            if mdl.startswith('s'):
                precision_ = [xx[mdl].precision_ for xx in x]
            else:
                precision_ = [xx.precision_ for xx in x]
            pr.append(np.array(np.abs(precision_)))
    
    pr = np.vstack(np.array(pr))
    pr[pr < 1e-3] = 0
    pp[mdl] = pr

In [ ]:
plt.close('all')
f, ax = plt.subplots(1, 1, figsize=(20,5), sharex=True)

results.plot_curve(trues_noshuffle, pp, ax=ax, fontsize=11, mode='roc', multiple_true=True)

In [ ]:
f

In [ ]:
reload(results)
plt.close('all')
f, ax = plt.subplots(1, 1, figsize=(20,10), sharex=True)

for i, n in enumerate(res_df.columns.levels[1]):
    pp = {}
    tt = ([np.abs(x['precs']) for x in datas[n]['data']])
    for mdl in mdls:
#         print(len(pp), len(tt))
        if mdl == 'ticc':
            
            obs_precs = np.array([ticc.precision_[l] for l in ticc.labels_])
        if mdl in ['ecov', 'gl']:
            pp[mdl] = [np.abs(np.array([xx.precision_ for xx in x]))
                       for x in vs[(mdl, n)]['model']]
        else:
            pp[mdl] = [np.abs(x[mdl].precision_ if mdl.startswith('s') else x.precision_)
                       for x in vs[(mdl, n)]['model']]
    results.plot_curve(tt, pp, ax=ax[i], fontsize=11, mode='roc', multiple_true=True)
f

In [ ]:
reload(results)
plt.close('all')
f, ax = plt.subplots(7, 1, figsize=(20,50), sharex=True)

for i, n in enumerate(res_df.columns.levels[1]):
    pp = {}
    tt = ([np.abs(x['precs']) for x in datas[n]['data']])
    for mdl in mdls:
#         print(len(pp), len(tt))
        if mdl == 'ticc': continue
        if mdl in ['ecov', 'gl']:
            pp[mdl] = [np.abs(np.array([xx.precision_ for xx in x]))
                       for x in vs[(mdl, n)]['model']]
        else:
            pp[mdl] = [np.abs(x[mdl].precision_ if mdl.startswith('s') else x.precision_)
                       for x in vs[(mdl, n)]['model']]
    results.plot_curve(tt, pp, ax=ax[i], fontsize=11, mode='roc', multiple_true=True)
f

In [ ]:
f.savefig('roc.pdf')